In [2]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Boas práticas para a criação de prompts

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Execute no Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      Veja no GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Execute no Vertex AI Workbench
    </a>
  </td>
</table>

## Visão geral

Este notebook aborda os fundamentos de design de prompts, incluindo algumas práticas recomendadas.

Saiba mais sobre o design de prompt na [documentação oficial](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-overview).

### Objetivo

Neste notebook, você aprenderá as práticas recomendadas sobre o design de prompts -- como projetar prompts para melhorar a qualidade de suas respostas.

Este notebook abrange as seguintes práticas recomendadas para engenharia imediata:

- Ser conciso
- Seja específico e com um texto bem definido
- Peça uma tarefa de cada vez
- Transforme tarefas generativas em tarefas de classificação
- Melhore a qualidade da resposta incluindo exemplos

### Custos
Este tutorial usa os seguintes componentes de Google Cloud:

* Vertex AI Generative AI Studio

Saiba mais sobre possíveis custos envolvidos [preços da Vertex AI](https://cloud.google.com/vertex-ai/pricing),
e use a [Calculadora de preços](https://cloud.google.com/products/calculator/)
para gerar uma estimativa de custo com base no uso projetado.

## Primeiros Passos

### Instalando os SDK da Vertex AI e da Cloud Translate API

In [3]:
# !pip install google-cloud-aiplatform --upgrade --user

**Somente Colab:** Descomente a célula a seguir para reiniciar o kernel ou use o botão para reiniciar o kernel.

In [4]:
# # Reinicia automaticamente o kernel após as instalações para que seu ambiente possa acessar os novos pacotes
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Autenticando seu ambiente de notebook
* Se você estiver usando o **Colab** para executar este notebook, descomente a célula abaixo e continue.
* Se você estiver usando o **Vertex AI Workbench**, confira as instruções de configuração [aqui](../setup-env/README.md).

In [5]:
# from google.colab import auth
# auth.authenticate_user()

### Importando as bibliotecas necessárias

**Somente Colab:** Descomente a célula a seguir para realizar o processo adequado de inicialização da SDK da Vertex AI.  

In [6]:
# import vertexai

# PROJECT_ID = "[seu-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [7]:
from vertexai.preview.language_models import TextGenerationModel

#### Carregando o modelo `text-bison`

In [8]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

## Boas práticas de design de prompts

A engenharia de prompt trata de como projetar seus prompts para que a resposta seja o que você realmente esperava ver.

A ideia de usar prompts "desagradáveis" é minimizar o ruído em seu prompt para reduzir a possibilidade de o LLM interpretar mal a intenção do prompt. Abaixo estão algumas diretrizes sobre como projetar prompts "desagradáveis".

Nesta seção, você abordará as seguintes práticas recomendadas quando a engenharia solicitar:

* Ser conciso
* Seja específico e com texto bem definido
* Peça uma tarefa de cada vez
* Melhore a qualidade da resposta incluindo exemplos
* Transforme tarefas generativas em tarefas de classificação para melhorar a segurança

### Seja conciso

🛑 Não recomendado. O prompt abaixo é desnecessariamente detalhado.

In [9]:
prompt = "Que nomes você acha que seriam interessantes para uma floricultura que se especializa mais em buquês de flores secas do que frescas? Muitíssimo brigado!"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

* **Flores Secas**
* **Flores Preservadas**
* **Flores Eternas**
* **Flores Intemporais**
* **Flores Atemporais**
* **Flores Inesquecíveis**
* **Flores Eternizadas**
* **Flores Para Sempre**
* **Flores que Duram**
* **Flores que Não Morrem**
* **Flores que Não Envelhecem**
* **Flores que Não Perdem a Cor**
* **Flores que Não Perdem o Brilho**
* **Flores que Não Perdem o Aroma**
* **Flores que Não Perdem a Beleza**
* **Flores que Não Perdem o Charme**
* **Flores que Não Perdem o Toque**
* **Flores que Não Perdem o Significado**
* **Flores que Não Perdem o Valor**
* **Flores que Não Perdem a Importância**
* **Flores que Não Perdem o Amor**


✅ Recomendado. O prompt abaixo é direto ao ponto e conciso.

In [10]:
prompt = "Sugira cinco nomes para uma floricultura que vende buquês de flores secas"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

* **Flores Secas**
* **Buquês Secos**
* **Flores Eternas**
* **Flores Preservadas**
* **Flores Desidratadas**


### Seja específico e escreva textos bem definidos

Suponha que você queira pensar em maneiras criativas de descrever a Terra.

🛑 Não recomendado. O prompt abaixo é muito genérico.

In [11]:
prompt = "Fale-me sobre a Terra"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

A Terra é o terceiro planeta do Sistema Solar, o único que se sabe ser habitado por seres vivos. É um planeta rochoso com uma atmosfera respirável e uma hidrosfera abundante. A Terra é o maior dos quatro planetas telúricos do Sistema Solar, e é o único com uma crosta oceânica contínua. A Terra é o único planeta conhecido que tem vida, e é o único planeta que foi visitado por seres humanos.

A Terra tem uma superfície dinâmica, com placas tectônicas que se movem constantemente. A atividade tectônica é responsável pela formação de montanhas, vulcões e terremotos. A Terra também tem uma atmosfera dinâmica, que é composta principalmente de nitrogênio e oxigênio. A atmosfera protege a Terra da radiação solar nociva e ajuda a manter a temperatura do planeta em um nível adequado para a vida.

A Terra tem uma hidrosfera abundante, que é composta de oceanos, mares, rios e lagos. A hidrosfera cobre cerca de 70% da superfície da Terra. A hidrosfera é importante para a vida na Terra, pois fornece 

✅ Recomendado. O prompt abaixo é específico e bem definido.

In [12]:
prompt = "Gere uma lista de motivos que fazem a Terra ser única comparada à outros planetas"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

* A Terra é o único planeta conhecido que abriga vida.
* A Terra tem uma atmosfera rica em oxigênio, que é essencial para a vida.
* A Terra tem água líquida em sua superfície, que é essencial para a vida.
* A Terra tem uma temperatura adequada para a vida, nem muito quente nem muito fria.
* A Terra tem um campo magnético que protege a vida da radiação solar.
* A Terra tem um tamanho e uma massa adequados para a vida.
* A Terra está localizada na zona habitável do Sistema Solar, o que significa que está a uma distância adequada do Sol para permitir a existência de água líquida na superfície.

Esses são apenas alguns dos motivos que tornam a Terra um planeta único e especial. É um lugar onde a vida pode florescer e se desenvolver, e é um lugar que devemos proteger.


### Peça uma tarefa de cada vez

🛑 Não recomendado. O prompt abaixo tem duas partes para a pergunta que pode ser feita separadamente.

In [13]:
prompt = "Qual é a melhor forma para ferver água e por que o céu é azul?"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

A melhor forma de ferver água é usando uma panela com fundo grosso e tampa. Coloque a água na panela e leve ao fogo médio. Quando a água começar a ferver, reduza o fogo para baixo e deixe ferver por mais alguns minutos. Desligue o fogo e deixe a água repousar por alguns minutos antes de usar.

O céu é azul porque a luz do Sol é composta de várias cores, incluindo o azul. Quando a luz do Sol atinge a atmosfera da Terra, as partículas de ar espalham as cores da luz em todas as direções. As partículas de ar espalham mais as cores da luz que têm comprimentos de onda mais curtos, como o azul. Por isso, vemos o céu azul.


✅ Recomendado. Os prompts abaixo solicitam uma tarefa por vez.

In [14]:
prompt = "Qual é a melhor forma para ferver água?"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

A melhor forma para ferver água é usando uma chaleira elétrica. As chaleiras elétricas são seguras, eficientes e fáceis de usar. Elas também são mais rápidas do que ferver água em uma panela no fogão.

Para usar uma chaleira elétrica, basta encher a chaleira com água até a marca indicada e ligar o aparelho. A chaleira começará a ferver a água em poucos minutos. Quando a água estiver fervendo, você ouvirá um sinal sonoro. Você pode então desligar o aparelho e despejar a água fervente em uma xícara ou caneca.

As chaleiras elétricas são uma ótima opção para quem quer ferver água rapidamente e com segurança. Elas são também uma ótima opção para quem quer economizar energia, pois são mais eficientes do que ferver água em uma panela no fogão.


In [15]:
prompt = "Por que o céu é azul?"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

O céu é azul porque a luz do Sol é espalhada pelas moléculas de ar na atmosfera. A luz do Sol é composta de todas as cores do arco-íris, mas as cores azuis são espalhadas mais do que as outras cores. Isso porque as moléculas de ar são menores do que as outras cores da luz do Sol, e elas interagem mais com as ondas de luz azuis. As outras cores da luz do Sol não são espalhadas tanto, então elas viajam para o nosso olho diretamente do Sol.

A quantidade de espalhamento da luz azul depende da quantidade de moléculas de ar na atmosfera. Quando o ar está mais limpo, há menos moléculas de ar para espalhar a luz azul, então o céu parece mais azul. Quando o ar está mais poluído, há mais moléculas de ar para espalhar a luz azul, então o céu parece mais cinza ou branco.

O ângulo do Sol também afeta a cor do céu. Quando o Sol está baixo no horizonte, a luz do Sol tem que viajar através de mais da atmosfera para chegar ao nosso olho. Isso significa que mais luz azul é espalhada, então o céu parec

### Cuidado com as alucinações

Embora os LLMs tenham sido treinados em uma grande quantidade de dados, eles podem gerar textos contendo declarações não fundamentadas na verdade ou na realidade; essas respostas do LLM são frequentemente chamadas de "alucinações" devido às suas capacidades limitadas de memorização. Observe que simplesmente solicitar que o LLM forneça uma citação não é uma solução para esse problema, pois há instâncias de LLMs que fornecem citações falsas ou imprecisas. 

Lidar com alucinações é um desafio fundamental dos LLMs e uma área de pesquisa em andamento, por isso é importante estar ciente de que os LLMs podem parecer dar a você declarações confiantes e corretas que, na verdade, são incorretas.

Observe que, se você pretende usar LLMs para os casos de uso criativo, alucinar pode ser bastante útil.

Tente o prompt como o abaixo repetidamente. Você pode notar que às vezes ele dirá com confiança, mas imprecisamente, "O primeiro elefante a visitar a lua foi Luna".

In [16]:
prompt = "Quem foi o primeiro elefante a visitar a lua?"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

O primeiro elefante a visitar a lua foi o elefante Babar. Babar foi um elefante criado pelo autor francês Jean de Brunhoff. Babar foi criado em 1931 e apareceu em vários livros e filmes. Babar é um elefante muito inteligente e corajoso que sempre está pronto para ajudar os outros. Ele é um grande amigo e um líder sábio. Babar é um personagem muito popular e tem sido amado por crianças de todo o mundo por gerações.


### Transforme tarefas generativas em tarefas de classificação para reduzir a variabilidade de saída

#### Tarefas generativas levam a uma maior variabilidade de saída

O prompt abaixo resulta em uma resposta aberta, útil para brainstorming, mas a resposta é altamente variável.

In [17]:
prompt = "Sou um estudante de ensino médio. Recomende-me atividades de programação para melhorar meus skills"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

* **Faça projetos pessoais.** Um ótimo jeito de aprender programação é fazer projetos pessoais que você se interesse. Isso pode ser qualquer coisa, desde um jogo simples até um aplicativo para celular. Quando você está trabalhando em um projeto que você se importa, você é mais propenso a ficar motivado e aprender mais.
* **Leia artigos e livros sobre programação.** Há muitos recursos disponíveis online e na biblioteca que podem ajudá-lo a aprender mais sobre programação. Leia artigos sobre tópicos que você está interessado, ou confira livros sobre programação que podem ajudá-lo a desenvolver suas habilidades.
* **Assista a vídeos sobre programação.** Há muitos vídeos excelentes sobre programação disponíveis no YouTube e em outros sites. Assistir a vídeos pode ser uma ótima maneira de aprender sobre novos conceitos e técnicas de programação.
* **Faça cursos online.** Existem muitos cursos online gratuitos e pagos que podem ajudá-lo a aprender programação. Alguns dos cursos mais populare

#### Tarefas de classificação reduzem a variabilidade de saída

O prompt abaixo resulta em uma escolha e pode ser útil se você quiser que a saída seja mais fácil de controlar.

In [18]:
prompt = """Sou um estudante de ensino médio. Quais destas atividades você recomenda e porquê:
a) Aprender Python
b) Aprender Javascript
c) Aprender Fortran
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

Eu recomendaria aprender Python. Python é uma linguagem de programação de alto nível, interpretada, de uso geral, com sintaxe fácil de ler e aprender. É uma linguagem multiparadigma, o que significa que pode ser usada para desenvolver programas de diferentes tipos, incluindo aplicativos de desktop, aplicativos móveis, servidores web e sistemas de big data. Python é também uma linguagem muito popular, sendo usada por empresas como Google, Facebook, Amazon e Netflix.

Javascript é outra linguagem de programação popular, mas é mais adequada para desenvolvimento web. Fortran é uma linguagem de programação mais antiga, que é principalmente usada para desenvolvimento de aplicações científicas e de engenharia.

Portanto, se você está procurando uma linguagem de programação que seja fácil de aprender e que possa ser usada para desenvolver uma variedade de programas, eu recomendaria Python.


### Melhore a qualidade da resposta incluindo exemplos

Outra maneira de melhorar a qualidade da resposta é adicionar exemplos em seu prompt. O LLM aprende no contexto dos exemplos sobre como responder. Normalmente, um a cinco exemplos (shots) são suficientes para melhorar a qualidade das respostas. Incluir muitos exemplos pode fazer com que o modelo ajuste demais os dados e reduza a qualidade das respostas.

Semelhante ao treinamento de modelo clássico, a qualidade e a distribuição dos exemplos são muito importantes. Escolha exemplos representativos dos cenários que você precisa que o modelo aprenda e mantenha a distribuição dos exemplos (por exemplo, número de exemplos por classe no caso de classificação) alinhada com sua distribuição real.

#### Zero-shot prompt

Abaixo está um exemplo de prompt zero-shot, onde você não fornece nenhum exemplo para o LLM dentro do próprio prompt.

In [19]:
prompt = """Decida se um Tweet apresenta um sentimento positivo, negativo ou neutro.

Tweet: Eu amei os seus videos mais recentes no YouTube!
Sentimento: 
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

positivo


#### One-shot prompt

Abaixo está um exemplo one-shot, onde você fornece um exemplo para o LLM dentro do prompt para fornecer alguma orientação sobre o tipo de resposta que você deseja.

In [20]:
prompt = """Decida se um Tweet apresenta um sentimento positivo, negativo ou neutro.

Tweet: Eu amei os seus videos mais recentes no YouTube!
Sentimento: positivo

Tweet: Foi difícil. Super chato 😠
Sentimento:
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

negativo


#### Few-shot prompt

Abaixo está um exemplo de few-shot, onde você fornece um exemplo para o LLM dentro do prompt para dar alguma orientação sobre o tipo de resposta que você deseja.

In [21]:
prompt = """Decida se um Tweet apresenta um sentimento positivo, negativo ou neutro.

Tweet: Eu amei os seus videos mais recentes no YouTube!
Sentimento: positivo

Tweet: Foi difícil. Super chato 😠
Sentimento: negativo

Tweet: Uma coisa me surpreendeu neste video - Ele foi realmente original. Ele não foi o mesmo velho conteúdo reciclado. Assista - você não se arrependerá.
Sentimento:
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

positivo


#### Escolhendo entre os métodos zero-shot, one-shot ou few-shot

Qual técnica de prompt usar dependerá exclusivamente do seu objetivo. Os prompts zero-shot são mais abertos e podem fornecer respostas criativas, enquanto os prompts one-shot e few-shot ensinam o modelo a se comportar para que você possa obter respostas mais previsíveis que sejam consistentes com os exemplos fornecidos.